In [2]:
# Load necessary libraries for data processing and calculations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from IPython.display import display, Math, Latex, Markdown, HTML

In [3]:
# Load Table 1: Wheatstone Bridge Data for Unknown Resistor (Calculating Unknown X(Ohmns))
wheatstone_bridge_unknown_x = pd.read_csv("../../data/lab04/Wheatstone_Bridge_PartA.csv")
wheatstone_bridge_unknown_x


,unknown_resistor,r_ohms,m_cm,n_cm,x_ohms
0,A,2700,49.7,50.3,NaN
1,B,1200,53.1,46.9,NaN
2,C,300,57.1,42.9,NaN
3,D,100,48.5,51.5,NaN
4,E,50,55.2,44.8,NaN
5,F,15,51.1,48.9,NaN


In [4]:
# Check the datatypes for each column
wheatstone_bridge_unknown_x.dtypes

unknown_resistor     object
r_ohms                int64
m_cm                float64
n_cm                float64
x_ohms              float64
dtype: object

In [5]:
# X_ohms column had null values and pandas treated it as Not A Number (NaN) data so we will manually
# fill those values with 0
wheatstone_bridge_unknown_x['x_ohms'] = wheatstone_bridge_unknown_x['x_ohms'].fillna(0)

# Convert m_cm and n_cm to meters
wheatstone_bridge_unknown_x['m_m'] = wheatstone_bridge_unknown_x['m_cm'] / 100
wheatstone_bridge_unknown_x['n_m'] = wheatstone_bridge_unknown_x['n_cm'] / 100

# Then filter the dataframe to remove columns m_cm and n_cm
wheatstone_bridge_unknown_x = wheatstone_bridge_unknown_x[['unknown_resistor', 'r_ohms',  'm_m', 'n_m', 'x_ohms']]
wheatstone_bridge_unknown_x




,unknown_resistor,r_ohms,m_m,n_m,x_ohms
0,A,2700,0.497,0.503,0.0
1,B,1200,0.531,0.469,0.0
2,C,300,0.571,0.429,0.0
3,D,100,0.485,0.515,0.0
4,E,50,0.552,0.448,0.0
5,F,15,0.511,0.489,0.0


In [6]:
# x_ohms is the resistance of the unknown resistor

from IPython.display import display, Math, Markdown

display(Markdown("**The fundamental equation for the Wheatstone bridge (when the galvanometer i₃ = 0) is:**"))


display(Math(r"X = \frac{R_3 R_1}{R_2}"))

display(Markdown("**In the slide-wire form of the Wheatstone Bridge,** the resistors "
                 "R₁ and R₂ are replaced by segments M and N of a uniform wire; "
                 "their resistance is proportional to their length."))

display(Math(r"\frac{X}{R} \;=\; \frac{M}{N}"))

display(Markdown("**Or, solving for X:**"))
display(Math(r"X = \frac{M R}{N}"))

wheatstone_bridge_unknown_x = wheatstone_bridge_unknown_x.copy()
# Compute X from measured lengths (ensure numeric and avoid division by zero)
# Coerce to numeric in case CSV parsed strings or blanks
wheatstone_bridge_unknown_x['m_m'] = pd.to_numeric(wheatstone_bridge_unknown_x.get('m_m', pd.Series(dtype=float)), errors='coerce')
wheatstone_bridge_unknown_x['n_m'] = pd.to_numeric(wheatstone_bridge_unknown_x.get('n_m', pd.Series(dtype=float)), errors='coerce')
wheatstone_bridge_unknown_x['r_ohms'] = pd.to_numeric(wheatstone_bridge_unknown_x.get('r_ohms', pd.Series(dtype=float)), errors='coerce')

# Optionally handle zeros/missing values in n_m to avoid division by zero
mask_valid = wheatstone_bridge_unknown_x['n_m'].notna() & (wheatstone_bridge_unknown_x['n_m'] != 0)

# Create x_ohms, leave invalid rows as <NA>
wheatstone_bridge_unknown_x.loc[mask_valid, 'x_ohms'] = (
    wheatstone_bridge_unknown_x.loc[mask_valid, 'm_m'] * wheatstone_bridge_unknown_x.loc[mask_valid, 'r_ohms']
) / wheatstone_bridge_unknown_x.loc[mask_valid, 'n_m']

# Round x_ohms column to 3 decimal places
wheatstone_bridge_unknown_x['x_ohms'] = wheatstone_bridge_unknown_x['x_ohms'].round(3)

# Show results (explicit display)
display(wheatstone_bridge_unknown_x.head())


**The fundamental equation for the Wheatstone bridge (when the galvanometer i₃ = 0) is:**

<IPython.core.display.Math object>

**In the slide-wire form of the Wheatstone Bridge,** the resistors R₁ and R₂ are replaced by segments M and N of a uniform wire; their resistance is proportional to their length.

<IPython.core.display.Math object>

**Or, solving for X:**

<IPython.core.display.Math object>

,unknown_resistor,r_ohms,m_m,n_m,x_ohms
0,A,2700,0.497,0.503,2667.793
1,B,1200,0.531,0.469,1358.635
2,C,300,0.571,0.429,399.301
3,D,100,0.485,0.515,94.175
4,E,50,0.552,0.448,61.607


In [7]:
# Load Table 2: Resistor Coded Values
color_coded_df = pd.read_csv('../../data/lab04/Wheatstone_Bridge_PartB.csv')
color_coded_df

,unknown_resistor,x_ohms,percent_error,color_coded_tolerance_%,within_tolerance_Yes_or_No
0,A,2700,NaN,10,NaN
1,B,1000,NaN,10,NaN
2,C,220,NaN,10,NaN
3,D,100,NaN,10,NaN
4,E,22,NaN,10,NaN
5,F,12,NaN,10,NaN


In [8]:
color_coded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   unknown_resistor            6 non-null      object 
 1   x_ohms                      6 non-null      int64  
 2   percent_error               0 non-null      float64
 3   color_coded_tolerance_%     6 non-null      int64  
 4   within_tolerance_Yes_or_No  0 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 372.0+ bytes


In [9]:
# Perform a couple transformations

# First convert the NaN values in the 'percent_error' column to 0
color_coded_df['percent_error'] = color_coded_df['percent_error'].fillna(0)



In [10]:
# Initialize the `within_tolerance_Yes_or_No` column as a pandas nullable boolean (<NA>/True/False)
# This keeps missing values explicit and avoids accidentally treating NaNs as True/False.
color_coded_df['within_tolerance_Yes_or_No'] = pd.Series(pd.NA, index=color_coded_df.index, dtype='boolean')

# Helper function to apply a tolerance check later when you decide the range.
# - tol_percent: numeric threshold (e.g., 5 for 5%)
# - column: the column to compare (default 'percent_error')
# - out_col: output boolean column name (default 'within_tolerance_Yes_or_No')
def apply_tolerance(df, tol_percent, column='percent_error', out_col='within_tolerance_Yes_or_No'):
    """Mark rows True where abs(column) <= tol_percent, False where > tol_percent.
    Missing values in `column` remain <NA> in the output (three-state logic).
    Returns the modified DataFrame.
    """
    # ensure numeric (non-numeric -> NaN)
    s = pd.to_numeric(df[column], errors='coerce')

    # prepare nullable boolean Series filled with <NA>
    result = pd.Series(pd.NA, index=df.index, dtype='boolean')

    # mask of rows with a valid numeric value
    mask = s.notna()

    # fill True/False for valid rows
    result.loc[mask] = (s.loc[mask].abs() <= tol_percent).astype('boolean')

    # assign back to dataframe
    df[out_col] = result
    return df

# Example: apply a 5% tolerance to show how it works (you can change or remove this)
# apply_tolerance(color_coded_df, 5)

# Show the first few rows of the relevant columns to confirm initialization
color_coded_df.head()

,unknown_resistor,x_ohms,percent_error,color_coded_tolerance_%,within_tolerance_Yes_or_No
0,A,2700,0.0,10,<NA>
1,B,1000,0.0,10,<NA>
2,C,220,0.0,10,<NA>
3,D,100,0.0,10,<NA>
4,E,22,0.0,10,<NA>


In [11]:
# Calculating the Percent Error
display(Math(r"\text{Percent Error} = \left(\frac{\left|X_{\text{expt}} - X_{\text{coded}}\right|}{X_{\text{coded}}}\right)\times 100\%"))

display(Markdown(""" Where:
- $X_{expt}$ is the calculated experimental resistance
- $X_{coded}$ is the color coded experimental value
"""))

# Next we'll merge the two tables together to make it easier to map the Percent Error calculations

merged_df = pd.merge(
    wheatstone_bridge_unknown_x,
    color_coded_df,
    on='unknown_resistor',
    how='outer'
)

# A bit of cleaning up to make these values easier to work with
merged_df = merged_df.drop(columns='x_ohms_y')
merged_df = merged_df.rename(columns={'x_ohms_x': 'R_expmt_ohms', 'r_ohms': 'R_coded_ohms'})

merged_df['percent_error'] = (merged_df['R_expmt_ohms'] - merged_df['R_coded_ohms']).abs() / merged_df['R_coded_ohms'] * 100
merged_df



<IPython.core.display.Math object>

 Where:
- $X_{expt}$ is the calculated experimental resistance
- $X_{coded}$ is the color coded experimental value


,unknown_resistor,R_coded_ohms,m_m,n_m,R_expmt_ohms,percent_error,color_coded_tolerance_%,within_tolerance_Yes_or_No
0,A,2700,0.497,0.503,2667.793,1.192852,10,<NA>
1,B,1200,0.531,0.469,1358.635,13.219583,10,<NA>
2,C,300,0.571,0.429,399.301,33.100333,10,<NA>
3,D,100,0.485,0.515,94.175,5.825000,10,<NA>
4,E,50,0.552,0.448,61.607,23.214000,10,<NA>
5,F,15,0.511,0.489,15.675,4.500000,10,<NA>


In [12]:
mask = merged_df['percent_error'].notna()
merged_df.loc[mask, 'within_tolerance_Yes_or_No'] = (
    merged_df.loc[mask, 'percent_error'] <= 10
).astype('boolean')

merged_df

,unknown_resistor,R_coded_ohms,m_m,n_m,R_expmt_ohms,percent_error,color_coded_tolerance_%,within_tolerance_Yes_or_No
0,A,2700,0.497,0.503,2667.793,1.192852,10,True
1,B,1200,0.531,0.469,1358.635,13.219583,10,False
2,C,300,0.571,0.429,399.301,33.100333,10,False
3,D,100,0.485,0.515,94.175,5.825000,10,True
4,E,50,0.552,0.448,61.607,23.214000,10,False
5,F,15,0.511,0.489,15.675,4.500000,10,True


In [ ]:
# Retrieve Data for Theoretical Resistances
theoretical_wire_resistances_df = pd.read_csv('../../data/lab04/Wheatstone_Bridge_PartC.csv')
theoretical_wire_resistances_df

,Spool,Length_L_of_wire(m),Wire_Material,Resistivity(Ωm),B&S_Gauge_#,Diameter_d(m),Cross_Sectional_Area_A(m²),Theoretical_Resistance_R(Ω)
0,2,10,Copper,1.680000e-08,28,0.000321,8.090000e-08,0.208
1,5,10,Copper-Nickle,4.900000e-07,22,0.000644,3.260000e-07,0.920


In [ ]:
# Retrieve Data for Experimenta Resistances
experimental_wire_resistances_df = pd.read_csv('../../data/lab04/Wheatstone_Bridge_PartD.csv')
experimental_wire_resistances_df

,Spool,R(Ω),M(m),N(m),X_Experimental(Ω)
0,2,1,0.533,0.467,1.14
1,5,10,0.455,0.545,8.35


In [29]:
# Get the dataframe for % Error 
percent_error_df = pd.read_csv ('../../data/lab04/Wheatstone_Bridge_PartE.csv')

# Our table is initially empty, however pandas marks the blank rows as NA values, we'll
# recategorize these values as 0
percent_error_df = percent_error_df.fillna(0)
percent_error_df

# Next we'll populate each field with data from our previous tables

# First, the theoretical resistances
percent_error_df['Theoretical_Resistance_R(Ω)'] = theoretical_wire_resistances_df['Theoretical_Resistance_R(Ω)']

# Next the experimental Resistances
percent_error_df['Experimental_Resistance_X(Ω)'] = experimental_wire_resistances_df['X_Experimental(Ω)']

# Now we can calculate the % error based on our values using the formula
display(Math(r"$\text{Percent Error} = (\frac{|I_{expt} - I_{theor}|}{I_{theor}}) * 100"))

percent_error_df['%_Error'] = (percent_error_df['Experimental_Resistance_X(Ω)'] - percent_error_df['Theoretical_Resistance_R(Ω)']).abs() / percent_error_df['Theoretical_Resistance_R(Ω)']
percent_error_df

<IPython.core.display.Math object>

,Spool,Theoretical_Resistance_R(Ω),Experimental_Resistance_X(Ω),%_Error
0,2,0.208,1.14,4.480769
1,5,0.920,8.35,8.076087


### Four possible sources of error in using the slide wire Wheatstone bridge are:
- An uneven uniformity in the slide wire itself could introduce variable resistances across the wire, leading to potential errors
- Parallax errors or misreading the position of the sliding contact could introduce errors if the we didn't properly position ourselves directly above the scale resulting in incorrect length measurements
- Temperature fluctuation could impact the resistance of both the wire and resistors, as resistance typically increases with temperature for metallic conductors.
- Contact resistance at the junctions and connections could introduce additional resistance resulting in inaccurate measurements

## Why it is important to set the sliding contact near the center of the bridge instead of near one of the ends
Setting the sliding contact near the center of the bridge is more accurate because it minimizes the effects of systematic errors and improves measurement sensitivity. When the contact is close to the center, the resistances on either side of the bridge wire are nearly equal, which means that a small movement of the contact produces a relatively large change in the galvanometer reading. This allows for finer adjustment and more precise detection of the balance point.

Additionally, the center of the wire is less affected by end effects, such as contact resistance at the terminals and non-uniformities near the ends of the wire. The wire is also more likely to be uniform in the middle, reducing errors due to variations in material or diameter. By balancing near the center, you ensure that the measurement is less sensitive to these sources of error, resulting in a more reliable and accurate determination of the unknown resistance.



## The effect of changes in temperature on the resistivity
A change in temperature has a significant effect on the resistivity of most materials. For metallic conductors, such as the wire used in a Wheatstone bridge, increasing the temperature generally causes the resistivity to increase. This is because, as temperature rises, the atoms in the metal vibrate more vigorously, which increases the likelihood that moving electrons will be scattered. This increased scattering impedes the flow of electric current, resulting in higher resistivity.

## The value of using the dry cell in operating the Wheatstone bridge
The value of the dry cell (its voltage) affects the operation of the Wheatstone bridge by determining the current that flows through the circuit and the sensitivity of the galvanometer. Using a higher voltage increases the current, which can make the galvanometer deflection more noticeable and the detection of the balance point easier. This can improve the precision of the measurement, especially if the galvanometer is not very sensitive.

However, there are limits to how large the voltage should be. If the voltage is too high, it can cause excessive current to flow through the resistors and the slide wire, leading to heating effects. This heating can change the resistance of the components due to temperature increases, introducing errors into the measurement. In extreme cases, it could even damage the resistors, the wire, or the galvanometer.

Therefore, while a moderate increase in voltage can improve sensitivity, there is a practical limit. The voltage should be high enough to give a clear galvanometer response but not so high that it causes significant heating or risks damaging the apparatus. In most laboratory Wheatstone bridge experiments, a voltage of 1.5 to 3 volts (typical of a single dry cell) is sufficient and safe.